In [ ]:
#필요한 라이브러리 정의
#!pip install gradio
import gradio as gr
#!pip install gtts

from tts_final import tts
#!pip install ultralytics
import torch
from ultralytics import YOLO
import cv2
import numpy as np

In [35]:
from collections import Counter
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("/content/best.pt")

# 객체 감지 결과를 딕셔너리로 변환하는 함수
def dictMaker(image):
    # 이미지 경로에서 객체 감지 실행
    results = model.predict(image_path)

    # 결과에서 객체 이름 추출
    detected_objects = []
    for result in results:
        detected_objects.extend([result.names[int(cls)] for cls in result.boxes.cls])

    # 객체의 개수 세기
    object_counts = Counter(detected_objects)

    return dict(object_counts)

# 예시 이미지 경로
image_path = "/content/KakaoTalk_20240620_183718962.jpg"

# dictMaker 함수 호출
object_counts = dictMaker(image_path)

# 결과 출력
print("Object Counts:")
print(object_counts)



image 1/1 /content/KakaoTalk_20240620_183718962.jpg: 800x608 1 paper, 2 plastics, 1 vinyl, 1319.8ms
Speed: 10.2ms preprocess, 1319.8ms inference, 1.3ms postprocess per image at shape (1, 3, 800, 608)
Object Counts:
{'plastic': 2, 'paper': 1, 'vinyl': 1}


In [36]:
# 영어로 설정된 class를 한국어로 번역 (tts를 위해서)
from collections import Counter

def wordChanger(word):
    if word == "can":
        return "캔"
    elif word == "glass":
        return "유리"
    elif word == "paper":
        return "종이"
    elif word == "pet":
        return "페트병"
    elif word == "plastic":
        return "플라스틱"
    elif word == "styrofoam":
        return "스티로폼"
    elif word == "vinyl":
        return "비닐"

In [39]:
# Gradio에서 동작을 수행하기 위한 함수 구현부

# 객체 감지 함수 구현
def objDetect(image):
  # 이미지 경로에서 객체 감지 실행
  results = model.predict(image)

  # 결과에서 객체 이름 추출
  detected_objects = []
  for result in results:
    detected_objects.extend([result.names[int(cls)] for cls in result.boxes.cls])

  # 객체의 개수 세기
  object_counts = Counter(detected_objects)
  object_counts_dict = dict(object_counts)

  # 최빈값 찾기
  most_common_word, _ = object_counts.most_common(1)[0]

  # 최빈값 번역하기
  best = wordChanger(most_common_word)

  return best

# 이미지에서 객체를 감지하고 TTS로 변환하는 함수
def process_image(image):
    detected_objects = objDetect(image)
    tts_result = tts(detected_objects)
    return tts_result

# Gradio 인터페이스 생성
with gr.Blocks(css="styles.css") as iface:
    # 타이틀 및 설명 텍스트 추가
    gr.Markdown("<div class='container'><h1>Voicena 보이시네</h1>")
    gr.Markdown("사진을 등록해주세요.</div>")

    with gr.Column():
        input_image = gr.Image(type="pil", label="Input Image")
        output_audio = gr.Audio(label="TTS Result", type="filepath", autoplay=True)
        submit_button = gr.Button("Process Image")

    submit_button.click(fn=process_image, inputs=input_image, outputs=output_audio)

    # 하단바 추가
    gr.Markdown("<div class='container'><hr>")
    gr.Markdown("© 2024 8 color bird. All rights reserved.</div>")

# 인터페이스 실행 및 공유 URL 생성
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9009976e574865caa1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [26]:
import gradio as gr
import tts_final  # tts_final.py 파일 임포트
from PIL import Image

# tts_final.py 파일 내용 불러오기
from tts_final import tts

# 객체 감지 함수 구현
def objDetect(image):
    """
    주어진 이미지에서 객체를 감지하는 함수.

    매개변수:
    image (ndarray): 감지할 이미지.

    반환:
    results (list): 감지된 객체의 결과 리스트.
    """
    # 예측 수행
    results = model.predict(image)
    print(results)
    print(bestResult(results))
    # results = ["can", "glass", "can", "paper", "can", "pet", "plastic", "styrofoam", "vinyl", "can"]

    return bestResult(results)

# 이미지에서 객체를 감지하고 TTS로 변환하는 함수
def process_image(image):
    detected_objects = objDetect(image)
    tts_result = tts(detected_objects)
    return tts_result, detected_objects  # TTS 결과와 감지된 객체 반환

# Gradio 인터페이스 생성
with gr.Blocks(css="styles.css") as iface:
    # 타이틀 및 설명 텍스트 추가
    gr.Markdown("<div class='container'><h1>Voicena 보이시네</h1>")
    gr.Markdown("사진을 등록해주세요.</div>")

    with gr.Column():
        input_image = gr.Image(type="pil", label="Input Image")
        #output_audio = gr.Audio(label="TTS Result", type="filepath", autoplay=True)
        output_text = gr.Textbox(label="Detected Objects")  # 감지된 객체 결과를 표시할 텍스트박스 추가
        submit_button = gr.Button("Process Image")

    submit_button.click(fn=process_image, inputs=input_image, outputs=[output_audio, output_text])  # 텍스트박스에 출력 추가

    # 하단바 추가
    gr.Markdown("<div class='container'><hr>")
    gr.Markdown("© 2023 Your Company Name. All rights reserved.</div>")

# 인터페이스 실행 및 공유 URL 생성
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8c6dc37586f0ed0ab0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
